In [ ]:
import psycopg2
import json
import os
from dotenv import load_dotenv
load_dotenv()
from sqlalchemy import create_engine

engine = create_engine(f"postgresql+psycopg2://{os.getenv('SQL_USER')}:{os.getenv('SQL_PASSWORD')}@{os.getenv('SQL_HOST')}:{os.getenv('SQL_PORT')}/{os.getenv('SQL_DATABASE')}")


In [ ]:
sql = """
SELECT json_build_object(
  'type', 'FeatureCollection',
  'features', json_agg(feature)
) AS geojson
FROM (
  SELECT json_build_object(
    'type','Feature',
    'id', coalesce(id, (row_number() OVER ())),
    'geometry', ST_AsGeoJSON(ST_Simplify(ST_Transform(geometry,4326), 0.01))::json,
    'properties', json_build_object(
        'id', id,
       'name', name,
       'name_normalized', name_normalized
    )
  ) AS feature
  FROM public.core_geozone
    WHERE id IN (
    select geozone_ptr_id from public.core_geodepartment where insee_code not like '97%'
  )
) AS f;
"""

sql = """
WITH metro AS (
  SELECT 
    cg.id,
    cg.name,
    cg.name_normalized,
    cgdep.insee_code,
    ST_Transform(cg.geometry, 4326) AS geom_translated
  FROM public.core_geozone cg
  LEFT JOIN public.core_geodepartment cgdep
    ON cg.id = cgdep.geozone_ptr_id
  WHERE cgdep.insee_code NOT LIKE '97%'
),
overseas AS (
  SELECT 
    cg.id,
    cg.name,
    cg.name_normalized,
    cgdep.insee_code,
    ST_Transform(cg.geometry, 4326) AS geom_4326,
    ST_Centroid(ST_Transform(cg.geometry, 4326)) AS geom_centroid
  FROM public.core_geozone cg
  LEFT JOIN public.core_geodepartment cgdep
    ON cg.id = cgdep.geozone_ptr_id
  WHERE cgdep.insee_code LIKE '97%' and not cgdep.insee_code = '973'
),
translated AS (
  SELECT 
    o.id,
    o.name,
    o.name_normalized,
    o.insee_code,
    ST_Translate(
      o.geom_4326,
      target.lon - ST_X(o.geom_centroid),
      target.lat - ST_Y(o.geom_centroid)
    ) AS geom_translated
  FROM overseas o
  JOIN (
    VALUES
      ('971', 51.0, -7.223655),
      ('972', 49.0, -7.223655),
      ('973', 47.0, -7.223655),
      ('974', 45.0, -7.223655),
      ('976', 43.0, -7.223655)
  ) AS target(insee_code, lat, lon)
    ON o.insee_code = target.insee_code
),
all_zones AS (
  SELECT * FROM metro
  UNION ALL
  SELECT * FROM translated
),
numbered AS (
  SELECT
    *,
    ROW_NUMBER() OVER () AS row_id
  FROM all_zones
)
SELECT json_build_object(
  'type', 'FeatureCollection',
  'features', json_agg(
    json_build_object(
      'type', 'Feature',
      'id', COALESCE(id, row_id),
      'geometry', ST_AsGeoJSON(ST_Simplify(geom_translated, 0.01))::json,
      'properties', json_build_object(
        'id', id,
        'name', name,
        'name_normalized', name_normalized,
        'insee_code', insee_code
      )
    )
  )
) AS geojson
FROM numbered;
"""

def export_geojson(path="core_geozone.geojson"):
    conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
    try:
        with conn.cursor() as cur:
            cur.execute(sql)
            row = cur.fetchone()
            geojson = row[0] if row else None
            if geojson is None:
                raise RuntimeError("No rows returned from query.")
        # Ensure pretty file
        with open(path, "w", encoding="utf-8") as fh:
            json.dump(geojson, fh, ensure_ascii=False, indent=2)
        print(f"Wrote {path}")
    finally:
        conn.close()

if __name__ == "__main__":
    export_geojson("core_geodepartment_v2.json")